In [63]:
# CURRENTLY WORKING IN THIS ONE
import os
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from PIL import Image
import numpy as np
from random import randint
from sklearn import preprocessing as p
import gc
import cv2
import math


In [64]:
torch.cuda.empty_cache()
gc.collect()

9

In [65]:
### DATA PREPROCESSING: NORMALIZE THE DATA ###

min_max_scaler = p.MinMaxScaler()
INPUT_DATA_PATH = 'test_input_new.npy'
OUTPUT_DATA_PATH = 'test_output_new.npy'
x_input = np.load(INPUT_DATA_PATH)
y_input = np.load(OUTPUT_DATA_PATH)

x_min = x_input.min(axis=(1, 2), keepdims=True)
x_max = x_input.max(axis=(1, 2), keepdims=True)
x_input = (x_input - x_min)/(x_max-x_min)

y_min = y_input.min(axis=(1, 2), keepdims=True)
y_max = y_input.max(axis=(1, 2), keepdims=True)
y_input = (y_input - y_min)/(y_max-y_min)

num_points = np.load('test_nums.npy')

In [66]:
SPLIT_IDX = 7500
x_train = torch.tensor(x_input[:SPLIT_IDX], dtype=torch.float).permute(0,3,1,2)
x_test = torch.tensor(x_input[SPLIT_IDX:], dtype=torch.float).permute(0,3,1,2)

y_train = torch.tensor(y_input[:SPLIT_IDX], dtype=torch.float).permute(0,3,1,2)
y_test = torch.tensor(y_input[SPLIT_IDX:], dtype=torch.float).permute(0,3,1,2)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
x_train = x_train.to(device)
x_test = x_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)

test_num_points = num_points[SPLIT_IDX:]

train_dataset = TensorDataset(x_train,y_train)

train_dataloader = DataLoader(train_dataset, batch_size=100, shuffle=True)

test_dataset = TensorDataset(x_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=100, shuffle=False)

75


In [67]:
# create a CNN
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        self.cnn_stack = nn.Sequential( 
            nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(5,5), stride=1, bias=True, padding=(2,2)), # input layer
            nn.ReLU(),

            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(7,7), stride=1, bias=True, padding=(3,3)),
            nn.ReLU(),

            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(9,9), stride=1, bias=True, padding=(4,4)), 
            nn.ReLU(),

            nn.Conv2d(in_channels=10, out_channels=1, kernel_size=(11,11), padding=(5,5)),
            nn.ReLU()
        )

    def forward(self, x):
        logits = self.cnn_stack(x)
        return logits

In [68]:
### PRE-TRAINED MODEL ###
model = torch.load('model_final_new_layers.pt')
if torch.cuda.is_available():
    model.cuda()

learning_rate = 3e-4
weight_decay=1e-5
optimizer = torch.optim.Adam(params=model.parameters(), weight_decay=weight_decay, lr=learning_rate)
batch_size = 16
epochs = 30
loss_fn = nn.MSELoss()
test_losses = []

In [72]:
def threshold_input(model_input):
    WHITE = 1
    BLACK = 0

    model_input[model_input != BLACK] = WHITE

def threshold_output(model_output):
    BLACK = 0
    THRESHOLD = 0.5
    model_output[model_output <= THRESHOLD] = BLACK

def mask(thresholded_input, model_output):
    return np.logical_or(thresholded_input, model_output)

In [70]:
def train_loop(dataloader, model, loss_fn, optimizer):
    avg_loss = 0
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        optimizer.zero_grad()
                  
        pred = model(X)
        loss = loss_fn(pred, y)
        avg_loss += loss
        loss.backward()
        optimizer.step()

    avg_loss = avg_loss / size
    print(f"average loss: {avg_loss}")

In [75]:
def test():
    pcnt_false_negatives = 0
    pcnt_false_positives = 0
    size = len(test_dataloader.dataset)
    i = 0
    with torch.no_grad():
        for data, target in test_dataloader:
            raw_output = model(data)
            # print(output*255)
            for out_i in range(100):
                input = np.array(data.cpu().data[out_i][0])
                output = np.array(raw_output.cpu().data[out_i][0])
                threshold_input(input)
                threshold_output(output)

                # mask the input onto the output to isolate the black points
                final_output = mask(input, output)
                # number of points the model counted
                num_total = np.count_nonzero(final_output == 0.0)
                # mask the target onto the final output to check 
                masked_target = mask(np.array(target.cpu().data[out_i])[0], final_output)
                # number of points IN THE LINE the model counted 
                num_in_line = np.count_nonzero(masked_target == 0.0)
                # num_expected = test_num_points[i]

                num_expected_in_line = test_num_points[i][0]
                num_expected_outside_line = test_num_points[i][1]

                # (expected positive - actual positive) / expected posiitive
                num_missed_in_line = num_expected_in_line - num_in_line
                pcnt_false_negatives += num_missed_in_line / num_expected_in_line


                # (actual negative) / expected posiitive
                num_identified_outside = num_total - num_in_line
                pcnt_false_positives += num_identified_outside / num_expected_outside_line

                # if (i == t):
                #     print(f"# false negatives: {num_missed_in_line}")
                #     print(f"# false positives: {num_identified_outside}")
                #     print(f"num expected in line: {num_expected_in_line}. num counted in line: {num_in_line}")
                #     print(f"num expected outside line: {num_expected_outside_line}. num counted outside line: {(num_expected_outside_line + num_expected_in_line) - num_in_line}")

                #     data_img = Image.fromarray(input*255)
                #     data_img.show()

                #     output_img = Image.fromarray(output*255)
                #     output_img.show()

                #     masked = Image.fromarray(final_output.astype(np.uint8)*255)
                #     masked.show()

                i += 1
    pcnt_false_negatives = pcnt_false_negatives / size
    pcnt_false_positives = pcnt_false_positives / size
    print(f"Average percent false negatives: {pcnt_false_negatives}")
    print(f"Average percent false positives: {pcnt_false_positives}")

In [71]:
### FOR TRAINING ONLY ###

# # test()
# for t in range(epochs):
#     print(f"Epoch {t+1}\n-------------------------------")
#     train_loop(train_dataloader, model, loss_fn, optimizer)
#     # test(500)
#     # test_loop(test_dataloader, model, loss_fn)
    
# print("Done!")

In [79]:
test()

Average percent false negatives: 0.06653022200022195
Average percent false positives: 0.010407142857142847


In [123]:
### DEBUGGING PURPOSES ONLY ###

i = randint(0,99)
i = 36
print (i)

with torch.no_grad():
    for data, target in test_dataloader:
        raw_output = model(data)    
        normOutput = (raw_output-raw_output.min())/(raw_output.max()-raw_output.min())

        target_arr = np.array(target.cpu().data[i][0])

        # INPUT
        input_arr = np.array(data.cpu().data[i][0])
        input = Image.fromarray(input_arr * 255)

        # OUTPUT
        output_arr = np.array(normOutput.cpu().data[i][0])
        output = Image.fromarray(output_arr * 255)
    
        threshold_input(input_arr)
        threshold_output(output_arr)

        input.show()
        output.show()

        masked_arr = mask(input_arr, output_arr)
        # print (masked_arr.astype(float))
        masked = Image.fromarray((masked_arr.astype(np.uint8) * 255))
        masked.show()

        num_dots = np.count_nonzero(masked_arr == 0.0)
        num_expected = test_num_points[i]


        target = Image.fromarray(target_arr*255)
        target.show()


        final_output = mask(input_arr, output_arr)
        # number of points the model counted
        # num_total = np.count_nonzero(final_output == 0.0)
        # mask the target onto the final output to check 
        masked_target = mask(target_arr, final_output)
        masked_target_img = Image.fromarray(masked_target.astype(np.uint8)*255)
        masked_target_img.show()
        
        #  number of points IN THE LINE the model counted 
        # num_in_line = np.count_nonzero(masked_target == 0.0)
        # num_expected = test_num_points[i]
        print(f"num counted: {num_dots}, num expected:{num_expected}")

        break

36
num counted: 6, num expected:[6. 4.]



(eog:10323): Gtk-WARNING **: 15:34:59.723: Could not load a pixbuf from icon theme.
This may indicate that pixbuf loaders or the mime database could not be found.


In [78]:
# torch.save(model, 'model_final_new_layers.pt')